# Database Administration

This notebook provides database management utilities:
- Initialize database
- Test connection
- View database statistics
- Cleanup utilities

In [1]:
from system import database, display
display.print_header("Database Administration")

In [2]:
display.print_subheader("Database Connection Test")

if database.test_connection():
    display.print_success("Database connection successful")
    
    # Show connection details
    from system.config import DB_CONFIG
    
    data = [
        ['Server', DB_CONFIG['host']],
        ['Database', DB_CONFIG['database']],
        ['Port', DB_CONFIG['port']],
        ['User', DB_CONFIG['user']]
    ]
    display.display_table(data, headers=['Property', 'Value'])
    
    # Get database version
    version = database.execute_scalar("SELECT version()")
    print(f"\nPostgreSQL Version:")
    print(version[:100] + "...")
    
else:
    display.print_error("Database connection failed")
    print("\nTroubleshooting:")
    print("1. Check if PostgreSQL container is running: docker ps")
    print("2. Verify environment variables in .env file")
    print("3. Try initializing the database (next cell)")

Property,Value
Server,postgres
Database,irp_db
Port,5432
User,irp_user



PostgreSQL Version:
PostgreSQL 15.14 on x86_64-pc-linux-musl, compiled by gcc (Alpine 14.2.0) 14.2.0, 64-bit...


In [8]:
display.print_subheader("Initialize Database")
print("Warning: This will recreate all tables!")
print("\nUncomment the next line to initialize:")
print("# success = db.init_database()")

# Uncomment to initialize:
#success = database.init_database()


Uncomment the next line to initialize:
# success = db.init_database()
Database initialized successfully


In [9]:
display.print_header("Database Statistics")

# Table row counts
tables = ['irp_cycle', 'irp_stage', 'irp_step', 'irp_step_run', 'irp_batch', 'irp_job']
stats = []

for table in tables:
    try:
        count = datbase.execute_scalar(f"SELECT COUNT(*) FROM {table}")
        stats.append([table, count])
    except:
        stats.append([table, 'N/A'])

display.display_table(stats, headers=['Table', 'Row Count'])

Table,Row Count
irp_cycle,N/A
irp_stage,N/A
irp_step,N/A
irp_step_run,N/A
irp_batch,N/A
irp_job,N/A


In [10]:
display.print_header("Database Statistics")

# Active cycle info
active = database.get_active_cycle()
if active:
    print(f"\nActive Cycle: {active['cycle_name']}")
else:
    display.print_warning("No active cycle")

In [11]:
display.print_header("Recent Step Runs")

query = """
SELECT
    c.cycle_name,
    CONCAT(sg.stage_num, '.', st.step_num) as step,
    st.step_name,
    sr.status,
    sr.started_ts,
    sr.completed_ts,
    EXTRACT(EPOCH FROM (sr.completed_ts - sr.started_ts)) as duration_seconds
FROM irp_step_run sr
INNER JOIN irp_step st ON sr.step_id = st.id
INNER JOIN irp_stage sg ON st.stage_id = sg.id
INNER JOIN irp_cycle c ON sg.cycle_id = c.id
ORDER BY sr.started_ts DESC
LIMIT 10
"""

df = database.execute_query(query)
if not df.empty:
    # Format duration
    df['duration'] = df['duration_seconds'].apply(display.format_duration)
    df = df.drop('duration_seconds', axis=1)
    
    display.display_dataframe(df, max_rows=10)
else:
    display.print_info("No step runs found")

# Cell 6: Cleanup Utilities

display.print_subheader("Cleanup Utilities")
print("Uncomment the operations you want to perform:\n")

print("# Clear failed step runs")
print("# rows = db.execute_command(\"DELETE FROM irp_step_run WHERE status = 'failed'\")")
print("# print(f'Deleted {rows} failed step runs')")
print()

print("# Clear all step runs but keep steps")
print("# rows = db.execute_command(\"DELETE FROM irp_step_run\")")
print("# print(f'Deleted {rows} step runs')")
print()

print("# Remove archived cycles older than 30 days")
print("# query = \"\"\"DELETE FROM irp_cycle ")
print("#           WHERE status = 'archived' ")
print("#           AND archived_ts < NOW() - INTERVAL '30 days'\"\"\"")
print("# rows = db.execute_command(query)")
print("# print(f'Deleted {rows} old archived cycles')")

# Cell 7: Query Builder

display.print_subheader("Custom Query")
print("Run custom SQL queries:\n")

query = "SELECT * FROM irp_cycle LIMIT 5"
print(f"Query: {query}\n")

try:
    df = database.execute_query(query)
    if not df.empty:
        display.display_dataframe(df)
    else:
        display.print_info("Query returned no results")
except Exception as e:
    display.print_error(f"Query failed: {str(e)}")

Uncomment the operations you want to perform:

# Clear failed step runs
# rows = db.execute_command("DELETE FROM irp_step_run WHERE status = 'failed'")
# print(f'Deleted {rows} failed step runs')

# Clear all step runs but keep steps
# rows = db.execute_command("DELETE FROM irp_step_run")
# print(f'Deleted {rows} step runs')

# Remove archived cycles older than 30 days
# query = """DELETE FROM irp_cycle 
#           WHERE status = 'archived' 
#           AND archived_ts < NOW() - INTERVAL '30 days'"""
# rows = db.execute_command(query)
# print(f'Deleted {rows} old archived cycles')


Run custom SQL queries:

Query: SELECT * FROM irp_cycle LIMIT 5



---